## Mediapipe를 이용한 제스쳐로 볼륨조절
_______________________
### 1. 라이브러리 추가하기

In [17]:
import cv2
import mediapipe as mp
import math
from ctypes import cast, POINTER
from comtypes import CLSCTX_ALL
from pycaw.pycaw import AudioUtilities, IAudioEndpointVolume

### 2. 오디오 장비 세팅하기
1. ```AudioUtilities``` 함수를 통해 speaker 장치 가져오기
2. 장치를 활성화하여 Audio의 끝 볼륨에 대한 정보를 가져오기
3. interface를 IAudioEndpointVolume의 포인터 정보를 자료형 변환해서 볼륨변환이 가능하도록

In [18]:
devices = AudioUtilities.GetSpeakers()
# print(devices)
# <POINTER(IMMDevice) ptr=0x1849c433640 at 18482d165c8>


interface = devices.Activate(IAudioEndpointVolume._iid_, CLSCTX_ALL,None)
# print(interface)
# <POINTER(IUnknown) ptr=0x1849c4e5f40 at 18482d166c8>

volume = cast(interface, POINTER(IAudioEndpointVolume))
# print(volume)
# <POINTER(IAudioEndpointVolume) ptr=0x1849c4e2ca0 at 18482d16648>

In [19]:
cap = cv2.VideoCapture(0)
#print(cap) # <VideoCapture 0000018485E3E4D0>

mpHands = mp.solutions.hands
# print(mp)
# <module 'mediapipe' from 'C:\\Users\\DONGJU\\AppData\\Roaming\\Python\\Python36\\site-packages\\mediapipe\\__init__.py'>
# print(mp.solutions)
# <module 'mediapipe.python.solutions' from ...\\mediapipe\\python\\solutions\\__init__.py
# print(mp.solutions.hands)
# <module 'mediapipe.python.solutions.hands' from ..\\mediapipe\\python\\solutions\\hands.py

my_hands = mpHands.Hands()
# <mediapipe.python.solutions.hands.Hands object at 0x00000184FDD44C18>

mpDraw = mp.solutions.drawing_utils
# ..\mediapipe\\python\\solutions\\drawing_utils.py

In [20]:
def dist(x1, y1, x2, y2):
    return math.sqrt(math.pow(x1-x2,2)) + math.sqrt(math.pow(y1-y2,2))

In [21]:
while True:
    success, img = cap.read()
    h,w,c = img.shape
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = my_hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            # 0번과 12번 길이가 0번과 10번 길이보다 긴 경우
            open = dist(handLms.landmark[0].x, handLms.landmark[0].y, 
                        handLms.landmark[12].x, handLms.landmark[12].y) > dist(handLms.landmark[0].x, handLms.landmark[0].y, 
                         handLms.landmark[10].x, handLms.landmark[10].y)
            # 0-14번 길이가 더 긴 경우
            if open == True:
                # 중지가 접히면 최대값, 펴지면 최솟값
                curdist = dist(handLms.landmark[0].x, handLms.landmark[0].y,
                            handLms.landmark[9].x, handLms.landmark[9].y) / (dist(handLms.landmark[0].x, handLms.landmark[0].y,
                                                                                  handLms.landmark[12].x, handLms.landmark[12].y))
                
                ratio = curdist
                print(1,ratio)
                
                
                curdist = -48 + (1-curdist)*36
                print(2,curdist)
                curdist = max(-48,curdist)
                
                
                if curdist < -10 and curdist >-48:
                    volume.SetMasterVolumeLevel(curdist,None)
                    year = int(1850+400*(ratio-0.39)/0.39)
                    print("year:",year)
                    
                    fname = "data/"+str(year)+".png"

                    if fname!="data/.png" and year<2100 and year>1850:
                        data = cv2.imread(fname,cv2.IMREAD_COLOR)
                        cv2.namedWindow("Map",flags=cv2.WINDOW_FREERATIO)
                        cv2.resizeWindow("Map",1920,1080)
                        cv2.imshow("Map",data)
                
                
                
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
    cv2.imshow("HandTracking",img)
    
    
        
    if cv2.waitKey(1)>0:
        break;
        
cv2.destroyAllWindows()
                

1 0.6723492182491873
2 -36.204571856970745
year: 2139
1 0.6819805237548506
2 -36.55129885517462
year: 2149
1 0.7046343185757977
2 -37.366835468728716
year: 2172
1 0.5689212684660199
2 -32.48116566477671
year: 2033
1 0.4925490551496712
2 -29.73176598538816
year: 1955
1 0.4898009840536803
2 -29.63283542593249
year: 1952
1 0.42968551143257344
2 -27.468678411572647
year: 1890
1 0.47360927529112923
2 -29.04993391048065
year: 1935
1 0.4783098695474094
2 -29.21915530370674
year: 1940
1 0.4826491047938069
2 -29.375367772577047
year: 1945
1 0.4658365364392688
2 -28.770115311813676
year: 1927
1 0.462553771410714
2 -28.651935770785702
year: 1924
1 0.463166089017526
2 -28.673979204630935
year: 1925
1 0.48935721466998955
2 -29.616859728119625
year: 1951
1 0.4779885910808923
2 -29.207589278912124
year: 1940
1 0.48888595149843833
2 -29.599894253943777
year: 1951
1 0.485462233918027
2 -29.476640421048973
year: 1947
1 0.47074390387972126
2 -28.946780539669962
year: 1932
1 0.4857526217453382
2 -29.48709

year: 1946
1 0.5513802683226776
2 -31.849689659616395
year: 2015
1 0.6073190433897947
2 -33.86348556203261
year: 2072
1 0.6475196552714705
2 -35.31070758977294
year: 2114
1 0.6796882142096884
2 -36.46877571154878
year: 2147
1 0.5877086106376492
2 -33.157509982955375
year: 2052
1 0.5521315666078369
2 -31.87673639788213
year: 2016
1 0.5092232375491207
2 -30.332036551768343
year: 1972
1 0.4505564051465641
2 -28.220030585276305
year: 1912
1 0.4429945270533547
2 -27.947802973920766
year: 1904
1 0.4464421848423125
2 -28.07191865432325
year: 1907
1 0.4516887172095111
2 -28.2607938195424
year: 1913
1 0.45836545413445284
2 -28.501156348840304
year: 1920
1 0.4576764202425549
2 -28.476351128731977
year: 1919
1 0.45779716812423954
2 -28.480698052472622
year: 1919
1 0.4600783354059401
2 -28.562820074613846
year: 1921
1 0.45776522482858306
2 -28.47954809382899
year: 1919
1 0.46138678186420107
2 -28.609924147111236
year: 1923
1 0.4737855404288684
2 -29.056279455439263
year: 1935
1 0.47872601650500457

1 0.5311078384024452
2 -31.119882182488027
year: 1994
1 0.5223654854330539
2 -30.805157475589944
year: 1985
1 0.5368247808453919
2 -31.325692110434108
year: 2000
1 0.5585425717293282
2 -32.107532582255814
year: 2022
1 0.5648662628177592
2 -32.33518546143933
year: 2029
1 0.5704818759451338
2 -32.53734753402482
year: 2035
1 0.5534816406507498
2 -31.925339063426993
year: 2017
1 0.5906341124893358
2 -33.26282804961609
year: 2055
1 0.5999615000576962
2 -33.59861400207706
year: 2065
1 0.5951563474101186
2 -33.42562850676427
year: 2060
1 0.6029272044251516
2 -33.705379359305454
year: 2068
1 0.6127843532833605
2 -34.060236718200976
year: 2078
1 0.6299814340700209
2 -34.67933162652075
year: 2096
1 0.6157680270391749
2 -34.167648973410294
year: 2081
1 0.6375322256682273
2 -34.95116012405619
year: 2103
1 0.6400523545555361
2 -35.0418847639993
year: 2106
1 0.6300824147900967
2 -34.682966932443485
year: 2096
1 0.6319871948863545
2 -34.75153901590876
year: 2098
1 0.6298059841256025
2 -34.67301542852

1 0.4444837234625059
2 -28.00141404465021
year: 1905
1 0.41997657608556394
2 -27.119156739080303
year: 1880
1 0.47098934953085464
2 -28.955616583110764
year: 1933
1 0.4435337038877595
2 -27.96721333995934
year: 1904
1 0.41689563310645705
2 -27.008242791832455
year: 1877
1 0.4108622353044938
2 -26.791040470961775
year: 1871
1 0.42140932358372707
2 -27.170735649014176
year: 1882
1 0.4424142972782689
2 -27.926914702017683
year: 1903
1 0.4872316571891781
2 -29.540339658810414
year: 1949
1 0.4596222591686783
2 -28.546401330072417
year: 1921
1 0.4568369478117843
2 -28.446130121224236
year: 1918
1 0.44286952738244795
2 -27.943302985768124
year: 1904
1 0.4424096477784883
2 -27.926747320025576
year: 1903
1 0.44193164035368065
2 -27.9095390527325
year: 1903
1 0.44894419787293904
2 -28.161991123425807
year: 1910
1 0.45401379325177893
2 -28.34449655706404
year: 1915
1 0.45447182281728266
2 -28.360985621422174
year: 1916
1 0.44522268907764273
2 -28.028016806795137
year: 1906
1 0.4730757301768519
2 

In [1]:
while True:
    success, img = cap.read()
    h,w,c = img.shape
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = my_hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            # 0번과 16번 길이가 0번과 14번 길이보다 긴 경우
            open = dist(handLms.landmark[0].x, handLms.landmark[0].y, 
                        handLms.landmark[14].x, handLms.landmark[14].y) < dist(handLms.landmark[0].x, handLms.landmark[0].y, 
                         handLms.landmark[16].x, handLms.landmark[16].y)
            # 0-14번 길이가 더 긴 경우
            if open == False:
                curdist = dist(handLms.landmark[4].x, handLms.landmark[4].y,
                            handLms.landmark[8].x, handLms.landmark[8].y) / (dist(handLms.landmark[2].x, handLms.landmark[2].y,
                                                                                  handLms.landmark[5].x, handLms.landmark[5].y) * 2)
                ratio = curdist
                print(1,ratio)
                
                
                curdist = -48 + (1-curdist)*36
                print(2,curdist)
                curdist = max(-48,curdist)
                
                
                if curdist < -10 and curdist >-48:
                    volume.SetMasterVolumeLevel(curdist,None)
                    
                    print("year:",1850+250*(ratio-0.39)/0.39)

                    fname = "data/"+str(1850+250*(ratio-0.39)/0.39)+".png"

                    if fname!="data/.png":
                        data = cv2.imread(fname,cv2.IMREAD_COLOR)
                        cv2.imshow("Map",data)
                
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
    cv2.imshow("HandTracking",img)
    if cv2.waitKey(1)>0:
        break;
        
cv2.destroyAllWindows()

NameError: name 'cap' is not defined

## 밑에는 제스처 포즈

In [17]:
import cv2
import mediapipe as np
import math

cap = cv2.VideoCapture(0)

myHands = mp.solutions.hands
my_hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

def dist(x1, y1, x2, y2):
    return math.sqrt(math.pow(x1-x2,2))+math.sqrt((math.pow(y1-y2,2)))

compareIndex = [[18,4],[6,8],[10,12],[14,16],[18,20]]
openIs = [False, False, False, False, False]
gesture = [[True, True, True, True, True, "Hi!"],
           [False, True, True, False, False, "Yeah!"],
           [True, True, False, False, True, "SpiderMan!"]]

while True:
    success, img = cap.read()
    h,w,c = img.shape
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = my_hands.process(imgRGB)
    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for i in range(0,5):
                open[i] = dist(handLms.landmark[0].x, handLms.landmark[0].y,
                                handLms.landmark[compareIndex[i][0]].x, handLms.landmark[compareIndex[i][0]].y) <
                            dist(handLms.landmark[0].x, handLms.landmark[0].y,
                                handLms.landmark[compareIndex[i][1]].x, handLms.landmark[compareIndex[i][1].y])
            print(open)
            text_x = handLms.landmark[0].x * w
            text_y = handLms.landmark[0].y * h
            for i in range(0,len(gesture)):
                flag = True
                for j in range(0,5):
                    if(gesture[i][j] != open[j]):
                        flag = False
                if flag==True:
                    cv2.putText(img.gesture[i][5],(round(text_x)-50, round(text_y)-250),
                                cv2.FONT_HERSHEY_PLAIN, 4,(0,0,0),4)
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
    
    cv2.imshow("HandTracking", img)
    if cv2.waitKey(1)>0:
        break;






UnboundLocalError: local variable 'myHand' referenced before assignment